In [1]:
import pandas as pd

df_procedure = pd.read_csv("procedureorderitems.csv", encoding="ISO-8859-1")
df_procedure

,admissionid,orderid,ordercategoryid,ordercategoryname,itemid,item,registeredat,registeredby
0,0,1994567,73,Opdr. Laboratorium afname,9267,Lab. CCH-VAT T = 2,27840000,ICV_IC-Verpleegkundig
1,0,1994708,73,Opdr. Laboratorium afname,9268,Lab. CCH-VAT T = 6,40500000,ICV_IC-Verpleegkundig
2,0,1994480,73,Opdr. Laboratorium afname,9269,Lab. CCH-VAT T = 0,22680000,ICV_IC-Verpleegkundig
3,0,1994479,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,22680000,ICV_IC-Verpleegkundig
4,0,1994566,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,27840000,ICV_IC-Verpleegkundig
...,...,...,...,...,...,...,...,...
2188621,23552,6290594,89,Opdr. Overig,15166,DOS-score invullen,16260000,ICV_MC-Verpleegkundig
2188622,23552,6291065,89,Opdr. Overig,15166,DOS-score invullen,44640000,ICV_MC-Verpleegkundig
2188623,23552,6290970,89,Opdr. Overig,15283,Verpl. beloop bijwerken,31740000,ICV_MC-Verpleegkundig
2188624,23552,6291245,90,Opdr. Lijnen/Catheter/Drains,12758,"Venflon doorspuiten NaCL0,9%",51960000,ICV_MC-Verpleegkundig


In [13]:
df_adm = pd.read_csv("admissions_filtered.csv", encoding="ISO-8859-1")
df = df_adm[df_adm['los_days'] > 0.25]

In [7]:
limit = 6 * 3600 * 1000  # 6 hours

df_procedure_filtered = df_procedure[(df_procedure['registeredat'] >= 0) & (df_procedure['registeredat'] <= limit)]
df_procedure_filtered

,admissionid,orderid,ordercategoryid,ordercategoryname,itemid,item,registeredat,registeredby
20,1,4703658,73,Opdr. Laboratorium afname,9267,Lab. CCH-VAT T = 2,9540000,ICV_IC-Verpleegkundig
22,1,4703548,73,Opdr. Laboratorium afname,9269,Lab. CCH-VAT T = 0,3420000,ICV_IC-Verpleegkundig
23,1,4703545,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,3360000,ICV_IC-Verpleegkundig
24,1,4703659,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,9540000,ICV_IC-Verpleegkundig
29,1,4703544,81,Opdr. Circulatie,7921,ECG,3360000,ICV_IC-Verpleegkundig
...,...,...,...,...,...,...,...,...
2188603,23552,6290518,83,Opdr. Neurologisch,8503,Neuro controles,11280000,ICV_MC-Verpleegkundig
2188604,23552,6290593,83,Opdr. Neurologisch,8503,Neuro controles,16200000,ICV_MC-Verpleegkundig
2188614,23552,6290592,83,Opdr. Neurologisch,19661,Pijnscore bepalen,16140000,ICV_MC-Verpleegkundig
2188621,23552,6290594,89,Opdr. Overig,15166,DOS-score invullen,16260000,ICV_MC-Verpleegkundig


In [33]:
df_procedure_filtered['ordercategoryname'].value_counts().head(30)

ordercategoryname
Opdr. Laboratorium afname       38319
Opdr. Neurologisch              17768
Opdr. Huid                      16032
Opdr. Circulatie                15042
Opdr. Overig                    14860
Opdr. Radiologie                 8364
Opdr. Kweken afnemen             5967
Opdr. Lijnen/Catheter/Drains     5170
1. Stoppen/Wijzigen              3359
Opdr. Respiratie                 1963
Opdr. Verzorging                 1139
Opdr. Psycho-sociaal             1013
Opdr. Houding & Beweging          514
Opdr. Gastro-Intestinaal          257
Opdr. Infectie                    161
Opdr. Urologisch                  152
Opdr. CVVH                        128
Opdr. Voeding                      36
Name: count, dtype: int64

In [41]:
top30 = df_procedure_filtered['ordercategoryname'].value_counts().head(20).index.tolist()

df_top30 = df_procedure_filtered[df_procedure_filtered['ordercategoryname'].isin(top30)]

valid_ids = df['admissionid'].unique()

df_top30_valid = df_top30[df_top30['admissionid'].isin(valid_ids)]

admission_counts_valid = (
    df_top30_valid
    .groupby('ordercategoryname')['admissionid']
    .nunique()                     
    .reindex(top30)                 
    .reset_index(name='n_admissions')
)

total_admissions = len(valid_ids)

admission_counts_valid['pct_admissions'] = (admission_counts_valid['n_admissions'] / total_admissions * 100).round(2)   
admission_counts_valid.sort_values(by='n_admissions', ascending=False)
admission_counts_valid

,ordercategoryname,n_admissions,pct_admissions
0,Opdr. Laboratorium afname,11083,61.66
1,Opdr. Neurologisch,6714,37.35
2,Opdr. Huid,12964,72.12
3,Opdr. Circulatie,10285,57.22
4,Opdr. Overig,9499,52.85
5,Opdr. Radiologie,7984,44.42
6,Opdr. Kweken afnemen,2619,14.57
7,Opdr. Lijnen/Catheter/Drains,3566,19.84
8,1. Stoppen/Wijzigen,1992,11.08
9,Opdr. Respiratie,1392,7.74


In [51]:
df_procedure_filtered['ordercategoryname'].value_counts()

ordercategoryname
Opdr. Laboratorium afname       38319
Opdr. Neurologisch              17768
Opdr. Huid                      16032
Opdr. Circulatie                15042
Opdr. Overig                    14860
Opdr. Radiologie                 8364
Opdr. Kweken afnemen             5967
Opdr. Lijnen/Catheter/Drains     5170
1. Stoppen/Wijzigen              3359
Opdr. Respiratie                 1963
Opdr. Verzorging                 1139
Opdr. Psycho-sociaal             1013
Opdr. Houding & Beweging          514
Opdr. Gastro-Intestinaal          257
Opdr. Infectie                    161
Opdr. Urologisch                  152
Opdr. CVVH                        128
Opdr. Voeding                      36
Name: count, dtype: int64

18 categories in total, we will create binary flags indicating if a patient had something from this category 

In [57]:
dummies = pd.get_dummies(
    df_procedure_filtered['ordercategoryname'],
    prefix='oc',         
    dtype=int
)

dummies['admissionid'] = df_procedure_filtered['admissionid']

flags = (
    dummies
      .groupby('admissionid')
      .max()
      .reset_index()
)

flags

,admissionid,oc_1. Stoppen/Wijzigen,oc_Opdr. CVVH,oc_Opdr. Circulatie,oc_Opdr. Gastro-Intestinaal,oc_Opdr. Houding & Beweging,oc_Opdr. Huid,oc_Opdr. Infectie,oc_Opdr. Kweken afnemen,oc_Opdr. Laboratorium afname,oc_Opdr. Lijnen/Catheter/Drains,oc_Opdr. Neurologisch,oc_Opdr. Overig,oc_Opdr. Psycho-sociaal,oc_Opdr. Radiologie,oc_Opdr. Respiratie,oc_Opdr. Urologisch,oc_Opdr. Verzorging,oc_Opdr. Voeding
0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
1,2,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,3,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
3,4,0,0,1,0,0,1,0,0,1,0,1,1,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19134,23548,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
19135,23549,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
19136,23550,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
19137,23551,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
flags = flags[flags['admissionid'].isin(valid_ids)].reset_index(drop=True)
flags

,admissionid,oc_1. Stoppen/Wijzigen,oc_Opdr. CVVH,oc_Opdr. Circulatie,oc_Opdr. Gastro-Intestinaal,oc_Opdr. Houding & Beweging,oc_Opdr. Huid,oc_Opdr. Infectie,oc_Opdr. Kweken afnemen,oc_Opdr. Laboratorium afname,oc_Opdr. Lijnen/Catheter/Drains,oc_Opdr. Neurologisch,oc_Opdr. Overig,oc_Opdr. Psycho-sociaal,oc_Opdr. Radiologie,oc_Opdr. Respiratie,oc_Opdr. Urologisch,oc_Opdr. Verzorging,oc_Opdr. Voeding
0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
1,2,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0
2,3,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
3,4,0,0,1,0,0,1,0,0,1,0,1,1,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17192,23545,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
17193,23546,0,0,0,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0
17194,23548,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
17195,23549,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0


## Merge with orginal dataframe


In [88]:
df_merged = pd.read_csv("df_adm2.csv", encoding="ISO-8859-1")
df_merged

,patientid,admissionid,location,admissionyeargroup,agegroup,weightgroup,heightgroup,specialty,los_days,ABP diastolisch_mean,...,Natrium-kalium-fosfaat_total_dose,Nitroglycerine (Nitro-pohl)_total_dose,Noradrenaline (Norepinefrine)_total_dose,Paracetamol_total_dose,Propofol (Diprivan)_total_dose,Ri-Lac (Ringers lactaat)_total_dose,SDD drank (4 x dgs)_total_dose,SDD pasta (4 x dgs)_total_dose,Thiamine (Vitamine B1)_total_dose,normaal_hartritme
0,0,0,IC,2003-2009,80+,60-69,160-169,Cardiochirurgie,1.750000,49.000000,...,0.0,0.5,0.0,0.0,100.00,0.0,0.0,0.0,0.0,1
1,1,1,IC,2010-2016,60-69,70-79,170-179,Cardiochirurgie,1.083333,59.500000,...,0.0,0.0,0.0,0.0,139.86,0.0,0.0,0.0,0.0,1
2,2,2,IC,2010-2016,60-69,90-99,180-189,Cardiochirurgie,0.958333,66.285714,...,0.0,0.0,1.6,1000.0,200.00,0.0,0.0,0.0,0.0,1
3,3,3,IC,2003-2009,50-59,90-99,180-189,Cardiochirurgie,0.958333,79.200000,...,0.0,0.5,0.0,0.0,800.00,0.0,0.0,0.0,0.0,1
4,4,4,IC&MC,2010-2016,70-79,70-79,170-179,Cardiochirurgie,2.083333,51.219780,...,0.0,0.0,0.0,1000.0,240.00,2000.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,20319,23545,MC,2010-2016,18-39,60-69,160-169,Traumatologie,5.458333,63.000000,...,0.0,0.0,0.0,1000.0,0.00,0.0,0.0,0.0,0.0,0
17882,20320,23546,IC,2010-2016,60-69,80-89,190+,Cardiologie,4.083333,73.310897,...,0.0,0.0,0.6,0.0,360.00,2000.0,10.0,1.0,0.0,1
17883,20322,23548,IC&MC,2010-2016,40-49,70-79,180-189,Neurochirurgie,11.000000,84.666667,...,0.0,0.0,0.7,1000.0,600.00,0.0,0.0,0.0,0.0,1
17884,20323,23549,MC,2010-2016,40-49,60-69,160-169,Nefrologie,0.666667,-1.000000,...,0.0,0.0,0.0,1000.0,0.00,0.0,0.0,0.0,0.0,1


In [90]:
df_merged2 = df_merged.merge(
    flags,           
    on='admissionid',
    how='left'      
)

# fill missing values of categories with 0 (as they have no value)
flag_cols = [c for c in df_merged2.columns if c.startswith('oc_')]
df_merged2[flag_cols] = df_merged2[flag_cols].fillna(0).astype(int)

df_merged2

,patientid,admissionid,location,admissionyeargroup,agegroup,weightgroup,heightgroup,specialty,los_days,ABP diastolisch_mean,...,oc_Opdr. Laboratorium afname,oc_Opdr. Lijnen/Catheter/Drains,oc_Opdr. Neurologisch,oc_Opdr. Overig,oc_Opdr. Psycho-sociaal,oc_Opdr. Radiologie,oc_Opdr. Respiratie,oc_Opdr. Urologisch,oc_Opdr. Verzorging,oc_Opdr. Voeding
0,0,0,IC,2003-2009,80+,60-69,160-169,Cardiochirurgie,1.750000,49.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,1,IC,2010-2016,60-69,70-79,170-179,Cardiochirurgie,1.083333,59.500000,...,1,0,0,1,0,1,0,0,0,0
2,2,2,IC,2010-2016,60-69,90-99,180-189,Cardiochirurgie,0.958333,66.285714,...,1,0,0,1,0,1,0,0,1,0
3,3,3,IC,2003-2009,50-59,90-99,180-189,Cardiochirurgie,0.958333,79.200000,...,1,0,0,0,1,1,0,0,0,0
4,4,4,IC&MC,2010-2016,70-79,70-79,170-179,Cardiochirurgie,2.083333,51.219780,...,1,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,20319,23545,MC,2010-2016,18-39,60-69,160-169,Traumatologie,5.458333,63.000000,...,1,0,0,0,0,0,0,0,0,0
17882,20320,23546,IC,2010-2016,60-69,80-89,190+,Cardiologie,4.083333,73.310897,...,1,0,1,1,0,0,1,0,0,0
17883,20322,23548,IC&MC,2010-2016,40-49,70-79,180-189,Neurochirurgie,11.000000,84.666667,...,0,0,1,0,0,0,0,0,0,0
17884,20323,23549,MC,2010-2016,40-49,60-69,160-169,Nefrologie,0.666667,-1.000000,...,1,0,0,1,0,0,0,0,0,0


### Process Data

In [19]:
df_process = pd.read_csv("processitems.csv", encoding="ISO-8859-1")
df_process

,admissionid,itemid,item,start,stop,duration
0,0,9159,Arterielijn Radialis,20520000,148800000,2138
1,0,9166,Trilumen Jugularis,20520000,148800000,2138
2,0,9174,Swan Ganz Jugularis,20520000,139020000,1975
3,0,9328,Beademen,20520000,69120000,810
4,0,9399,Wonddrain 1,20520000,94320000,1230
...,...,...,...,...,...,...
256710,23551,9422,Perifeer infuus,3900000,165660000,2696
256711,23551,12634,Tube,3900000,165660000,2696
256712,23551,13009,Parenchymdrain,3960000,165660000,2695
256713,23552,9422,Perifeer infuus,60000,64980000,1082


In [21]:
limit = 6 * 3600 * 1000  # 6 hours

df_process_filtered = df_process[(df_process['start'] >= 0) & (df_process['start'] <= limit)]
df_process_filtered

,admissionid,itemid,item,start,stop,duration
0,0,9159,Arterielijn Radialis,20520000,148800000,2138
1,0,9166,Trilumen Jugularis,20520000,148800000,2138
2,0,9174,Swan Ganz Jugularis,20520000,139020000,1975
3,0,9328,Beademen,20520000,69120000,810
4,0,9399,Wonddrain 1,20520000,94320000,1230
...,...,...,...,...,...,...
256710,23551,9422,Perifeer infuus,3900000,165660000,2696
256711,23551,12634,Tube,3900000,165660000,2696
256712,23551,13009,Parenchymdrain,3960000,165660000,2695
256713,23552,9422,Perifeer infuus,60000,64980000,1082


In [29]:
top30 = df_process_filtered['item'].value_counts().head(60).index.tolist()

df_top30 = df_process_filtered[df_process_filtered['item'].isin(top30)]

valid_ids = df['admissionid'].unique()

df_top30_valid = df_top30[df_top30['admissionid'].isin(valid_ids)]

admission_counts_valid = (
    df_top30_valid
    .groupby('item')['admissionid']
    .nunique()                     
    .reindex(top30)                 
    .reset_index(name='n_admissions')
)

total_admissions = len(valid_ids)

admission_counts_valid['pct_admissions'] = (admission_counts_valid['n_admissions'] / total_admissions * 100).round(2)   
admission_counts_valid.sort_values(by='n_admissions', ascending=False)
admission_counts_valid

,item,n_admissions,pct_admissions
0,Perifeer infuus,17069,94.96
1,Urine-Catheter (CAD),15469,86.06
2,Arterielijn Radialis,14645,81.47
3,Beademen,11949,66.48
4,Sonde,10861,60.42
5,Tube,10236,56.95
6,Trilumen Jugularis,8686,48.32
7,Thoraxdrain 1,8157,45.38
8,Swan Ganz Jugularis,5935,33.02
9,Wonddrain 1,4563,25.39


We will only use the items that have a percentage higher than 5.38, excluding items that state 'niet meer gebruiken' (do not use)

In [92]:
items = [
    "Perifeer infuus",
    "Urine-Catheter (CAD)",
    "Arterielijn Radialis",
    "Beademen",
    "Sonde",
    "Tube",
    "Trilumen Jugularis",
    "Thoraxdrain 1",
    "Swan Ganz Jugularis",
    "Wonddrain 1",
    "Sheet Swan Ganz",
    "Pacemakerdraden",
    "Opwarmen (Actief uitwendig)",
    "Wonddrain 2",
    "Pacemaker extern"
]

df_proc_keep = df_process_filtered.loc[
    df_process_filtered['admissionid'].isin(valid_ids) &
    df_process_filtered['item'].isin(items),
    ['admissionid', 'item']].copy()

dummies = pd.get_dummies(
    df_proc_keep['item'],
    prefix='process',
    dtype=int
)

dummies['admissionid'] = df_proc_keep['admissionid']

flags_proc = (
    dummies
      .groupby('admissionid')
      .max()
      .reset_index()
)

flags_proc


,admissionid,process_Arterielijn Radialis,process_Beademen,process_Opwarmen (Actief uitwendig),process_Pacemaker extern,process_Pacemakerdraden,process_Perifeer infuus,process_Sheet Swan Ganz,process_Sonde,process_Swan Ganz Jugularis,process_Thoraxdrain 1,process_Trilumen Jugularis,process_Tube,process_Urine-Catheter (CAD),process_Wonddrain 1,process_Wonddrain 2
0,0,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1
1,1,1,1,0,0,0,1,0,1,1,1,0,1,1,0,0
2,2,1,1,0,0,0,1,1,1,1,1,1,1,1,0,0
3,3,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1
4,4,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,23545,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
17584,23546,1,1,0,0,0,1,0,0,0,0,0,1,1,0,0
17585,23548,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0
17586,23549,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [94]:
df_final = df_merged2.merge(
    flags_proc,
    on='admissionid',
    how='left'
)

# fill NaN
proc_flag_cols = [c for c in df_final.columns if c.startswith('process_')]
df_final[proc_flag_cols] = df_final[proc_flag_cols].fillna(0).astype(int)

df_final

,patientid,admissionid,location,admissionyeargroup,agegroup,weightgroup,heightgroup,specialty,los_days,ABP diastolisch_mean,...,process_Perifeer infuus,process_Sheet Swan Ganz,process_Sonde,process_Swan Ganz Jugularis,process_Thoraxdrain 1,process_Trilumen Jugularis,process_Tube,process_Urine-Catheter (CAD),process_Wonddrain 1,process_Wonddrain 2
0,0,0,IC,2003-2009,80+,60-69,160-169,Cardiochirurgie,1.750000,49.000000,...,1,0,1,1,1,1,1,1,1,1
1,1,1,IC,2010-2016,60-69,70-79,170-179,Cardiochirurgie,1.083333,59.500000,...,1,0,1,1,1,0,1,1,0,0
2,2,2,IC,2010-2016,60-69,90-99,180-189,Cardiochirurgie,0.958333,66.285714,...,1,1,1,1,1,1,1,1,0,0
3,3,3,IC,2003-2009,50-59,90-99,180-189,Cardiochirurgie,0.958333,79.200000,...,1,0,1,1,1,1,1,1,1,1
4,4,4,IC&MC,2010-2016,70-79,70-79,170-179,Cardiochirurgie,2.083333,51.219780,...,1,1,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,20319,23545,MC,2010-2016,18-39,60-69,160-169,Traumatologie,5.458333,63.000000,...,1,0,0,0,0,0,0,1,0,0
17882,20320,23546,IC,2010-2016,60-69,80-89,190+,Cardiologie,4.083333,73.310897,...,1,0,0,0,0,0,1,1,0,0
17883,20322,23548,IC&MC,2010-2016,40-49,70-79,180-189,Neurochirurgie,11.000000,84.666667,...,1,0,1,0,0,0,1,0,0,0
17884,20323,23549,MC,2010-2016,40-49,60-69,160-169,Nefrologie,0.666667,-1.000000,...,1,0,0,0,0,0,0,1,0,0


In [96]:
df_final['los'] = df_final.pop('los')

df_final

,patientid,admissionid,location,admissionyeargroup,agegroup,weightgroup,heightgroup,specialty,los_days,ABP diastolisch_mean,...,process_Sheet Swan Ganz,process_Sonde,process_Swan Ganz Jugularis,process_Thoraxdrain 1,process_Trilumen Jugularis,process_Tube,process_Urine-Catheter (CAD),process_Wonddrain 1,process_Wonddrain 2,los
0,0,0,IC,2003-2009,80+,60-69,160-169,Cardiochirurgie,1.750000,49.000000,...,0,1,1,1,1,1,1,1,1,1
1,1,1,IC,2010-2016,60-69,70-79,170-179,Cardiochirurgie,1.083333,59.500000,...,0,1,1,1,0,1,1,0,0,1
2,2,2,IC,2010-2016,60-69,90-99,180-189,Cardiochirurgie,0.958333,66.285714,...,1,1,1,1,1,1,1,0,0,0
3,3,3,IC,2003-2009,50-59,90-99,180-189,Cardiochirurgie,0.958333,79.200000,...,0,1,1,1,1,1,1,1,1,0
4,4,4,IC&MC,2010-2016,70-79,70-79,170-179,Cardiochirurgie,2.083333,51.219780,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,20319,23545,MC,2010-2016,18-39,60-69,160-169,Traumatologie,5.458333,63.000000,...,0,0,0,0,0,0,1,0,0,1
17882,20320,23546,IC,2010-2016,60-69,80-89,190+,Cardiologie,4.083333,73.310897,...,0,0,0,0,0,1,1,0,0,1
17883,20322,23548,IC&MC,2010-2016,40-49,70-79,180-189,Neurochirurgie,11.000000,84.666667,...,0,1,0,0,0,1,0,0,0,1
17884,20323,23549,MC,2010-2016,40-49,60-69,160-169,Nefrologie,0.666667,-1.000000,...,0,0,0,0,0,0,1,0,0,0


In [98]:
df_final2 = df_final.drop(['patientid', 'location', 'los_days'], axis=1)
df_final2

,admissionid,admissionyeargroup,agegroup,weightgroup,heightgroup,specialty,ABP diastolisch_mean,ABP gemiddeld_mean,ABP systolisch_mean,Act.HCO3 (bloed)_mean,...,process_Sheet Swan Ganz,process_Sonde,process_Swan Ganz Jugularis,process_Thoraxdrain 1,process_Trilumen Jugularis,process_Tube,process_Urine-Catheter (CAD),process_Wonddrain 1,process_Wonddrain 2,los
0,0,2003-2009,80+,60-69,160-169,Cardiochirurgie,49.000000,68.000000,102.000000,23.600000,...,0,1,1,1,1,1,1,1,1,1
1,1,2010-2016,60-69,70-79,170-179,Cardiochirurgie,59.500000,74.833333,114.333333,25.600000,...,0,1,1,1,0,1,1,0,0,1
2,2,2010-2016,60-69,90-99,180-189,Cardiochirurgie,66.285714,82.714286,122.714286,22.633333,...,1,1,1,1,1,1,1,0,0,0
3,3,2003-2009,50-59,90-99,180-189,Cardiochirurgie,79.200000,96.400000,130.800000,22.649999,...,0,1,1,1,1,1,1,1,1,0
4,4,2010-2016,70-79,70-79,170-179,Cardiochirurgie,51.219780,66.474265,111.043956,21.925001,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17881,23545,2010-2016,18-39,60-69,160-169,Traumatologie,63.000000,82.666667,124.000000,21.650001,...,0,0,0,0,0,0,1,0,0,1
17882,23546,2010-2016,60-69,80-89,190+,Cardiologie,73.310897,92.403175,132.128205,19.750000,...,0,0,0,0,0,1,1,0,0,1
17883,23548,2010-2016,40-49,70-79,180-189,Neurochirurgie,84.666667,102.666667,124.500000,21.649999,...,0,1,0,0,0,1,0,0,0,1
17884,23549,2010-2016,40-49,60-69,160-169,Nefrologie,-1.000000,-1.000000,-1.000000,20.299999,...,0,0,0,0,0,0,1,0,0,0


In [100]:
df_final2.to_csv('df_final2.csv', index=False)


In [104]:
pd.set_option('display.max_rows', None)

In [106]:
df_final2.isna().sum().sort_values()


admissionid                                         0
oc_Opdr. Infectie                                   0
oc_Opdr. Huid                                       0
oc_Opdr. Houding & Beweging                         0
oc_Opdr. Gastro-Intestinaal                         0
oc_Opdr. Circulatie                                 0
oc_Opdr. CVVH                                       0
oc_1. Stoppen/Wijzigen                              0
normaal_hartritme                                   0
process_Wonddrain 2                                 0
oc_Opdr. Kweken afnemen                             0
oc_Voeding Vast voedsel                             0
oc_Voeding Enteraal oud                             0
oc_Voeding Enteraal                                 0
oc_Voeding Drinken                                  0
oc_Research                                         0
oc_Niet iv Zalven/CrÃ¨mes/Druppels                  0
oc_Niet iv Tractus Respiratorius                    0
oc_Niet iv Tractus Digestivu